<font size="+3">Topography (raster)</font>

Handles Elevation raster maps (DEM). Currently manages the SRTM files of NASA (30m resolution). 

Import the toolkit:

In [1]:
from hera import toolkitHome

tk = toolkitHome.getToolkit(toolkitName=toolkitHome.GIS_RASTER_TOPOGRAPHY)

Check for Data Sources loaded in the Project:

In [2]:
tk.getDataSourceDocument("SRTMGL1")

<Measurements: {
    "_cls": "Metadata.Measurements",
    "projectName": "DOCUMENTATION",
    "desc": {
        "desc": {},
        "toolkit": "TopographyToolkit",
        "datasourceName": "SRTMGL1",
        "version": [
            0,
            0,
            1
        ]
    },
    "type": "ToolkitDataSource",
    "resource": "/home/salo/development/hera/hera/doc/jupyter/data/raster/N31E034.SRTMGL1.hgt",
    "dataFormat": "string"
}>

Check for default data source of project:

In [3]:
tk.getConfig()['defaultSRTM']

'SRTMGL1'

### Elevation calculation

For calculating an elevation in a specific point, one can use **getPointElevation()**.  
Arguments:
- lat : Latitiute in the WSG projection.
- lon : Longitude in the WSG projection.
- dataSourceName: The name of the data source to use. If not provided, will use the default datasource.

In [4]:
lat = 32.095617
lon = 34.772145
tk.getPointElevation(lat,lon)

5.056800000003477

### Create an Elevation Mesh 
For creating an elevation mesh, one can use the **getElevation()** function.  
Arguments:
- minx : Minimum value of X axis. If using WSG84 - the minimum Latitude coodinates.
- miny : Minimum value of Y axis. If using WSG84 - the minimum Longitude coodinates.
- maxx :  Maximum value of X axis. If using WSG84 - the minimum Latitude coodinates.
- maxy :  Maximum value of Y axis. If using WSG84 - the minimum Longitude coodinates.
- dxdy : Spatial resolution of the output land cover map. Defines the step size for the grid points
  in both the x (longitude) and y (latitude) directions within the specified bounding box. Default=30.
- inputCRS: The ESPG of the coordinates. Default is set to WSG84.
- dataSourceName: The name of the data source to use. If not provided, will use the default datasource.
- LandcoverDataSource: The name of the Landcover data source to use.  If not provided, will use the default datasource.

The function will return an xarray with the elevation values with the landcover values in each cell.

Let's use an example with North Tel Aviv coordinates:

In [5]:
minlat, minlon = 32.095617,34.772145
maxlat, maxlon  = 32.116561,34.844663
dxdy = 100

In [6]:
elevation = tk.getElevation(minlat,minlon,maxlat,maxlon,dxdy)
elevation

Warning 1: Starting with GDAL 3.7, PIXELTYPE=SIGNEDBYTE is no longer used to signal signed 8-bit raster. Change your code to test for the new GDT_Int8 data type instead.


<xarray.DataArray (i: 69, j: 23)> Size: 2kB
array([[ 0,  0,  0, ..., 15, 15, 15],
       [ 0,  0,  0, ..., 15, 15, 15],
       [ 0,  0,  0, ..., 15, 15, 15],
       ...,
       [13, 13, 13, ..., 13, 13, 13],
       [13, 13, 13, ..., 13, 13, 13],
       [13, 13, 13, ..., 13, 13, 13]], dtype=uint8)
Coordinates:
  * i          (i) int64 552B 0 1 2 3 4 5 6 7 8 9 ... 60 61 62 63 64 65 66 67 68
  * j          (j) int64 184B 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22
    lat        (i, j) float64 13kB 32.12 32.11 32.11 32.11 ... 32.1 32.1 32.1
    lon        (i, j) float64 13kB 34.77 34.77 34.77 34.77 ... 34.84 34.84 34.84
    landcover  (i, j) uint8 2kB 0 0 0 0 0 0 0 0 0 ... 13 13 13 13 13 13 13 13 13
    dxdy       int64 8B 100
    elevation  (i, j) float64 13kB 0.0 0.0 0.0 0.0 ... 13.66 12.97 12.64 16.92
Attributes:
    landcover_description:  {}

If you already have an mesh xarray of a specific region, you can use **getElevationFromLandcover()**, and it will add an elevation field in the xarray.  
Arguments:
- xarray : The landcover xarray to add elevation to.
- dataSourceName: The name of the data source to use. If not provided, will use the default datasource.

### Create an STL file (Python)
You can create an STL string of a specified region in your python script using **createElevationSTL()** function.  
Arguments:
- minx : Minimum value of X axis. If using WSG84 - the minimum Latitude coodinates.
- miny : Minimum value of Y axis. If using WSG84 - the minimum Longitude coodinates.
- maxx : Maximum value of X axis. If using WSG84 - the minimum Latitude coodinates.
- maxy : Maximum value of Y axis. If using WSG84 - the minimum Longitude coodinates.
- dxdy : Spatial resolution of the output land cover map. Defines the step size for the grid points in both the x (longitude) and y (latitude) directions within the specified bounding box. Default=30.
- shiftx: Used when one wants to set another point as origin center. Default=0.
- shifty: Used when one wants to set another point as origin center. Default=0.
- inputCRS: The ESPG of the coordinates. Default is set to WSG84.
- dataSourceName: The name of the data source to use. If not provided, will use the default datasource.
- solidName: The name of the topography solid. Default="Topography".

In [12]:
stlstr = tk.createElevationSTL(minlat,minlon,maxlat,maxlon,dxdy)

Warning 1: Starting with GDAL 3.7, PIXELTYPE=SIGNEDBYTE is no longer used to signal signed 8-bit raster. Change your code to test for the new GDT_Int8 data type instead.


If you dont want to create a new xarray and just create an STL string from an existing xarray, use **getElevationSTL()**.  
You just need to pass the xarray to the function:


In [10]:
stlstr = tk.getElevationSTL(elevation)

Later, we can save the string as an STL file with the lines below:

In [11]:
with open("North_Tel_Aviv.stl", "w") as stl_file:
    stl_file.write(stlstr)

### Create STL in CLI  
You can also create a STL file using the CLI:  

<p style="background:black">
<code style="background:black;color:white">>> hera-GIS topography_raster toSTL --minx --miny --maxx --maxy --dxdy --inputCRS --outputCRS --dataSourceName --fileName --projectName
</code>
</p>   
  
Arguments:  
- **minx**: Lower x boundingbox (float).
- **miny**: Lower y boundingbox (float).
- **maxx**: Upper x boundingbox (float).
- **maxy**: Upper y boundingbox (float).
- **dxdy**: The resolution of the mesh, Default is 30m (float).
- **inputCRS**: The CRS of the input coordinates. To use WSG84, pass 4326, for ITM pass 2039. The default is WSG84 (int).
- **solidName**: The name of the topography solid. Default='Topography".
- **dataSourceName**: The name of the datasource. If not supplied, uses the default datasource of the toolkit (str). 
- **fileName**: Path to save STL file (str).
- **projectName**: The Project Name. If not provided, uses default project (str).

Before applying the function, please make sure the datasource (or a default datasource) is loaded to the project you supply.  
For this:

1) Add the repository to the repository list:
   
   ```hera-project repository add [PATH_TO_REPOSITORY_WITH_DATASOURCE]```
   
3) Load the repository to Project:
   If the project is new, create a new project:
   
   ```hera-project project create [PROJECT_NAME]```  
   
   If it is an existing project, load it to the project:
   
   ```hera-project project loadRepositories [PATH_TO_REPOSITORY_WITH_DATASOURCE]```


For more details on repositories and project, read :  [Projects](../../../../datalayer/Project.ipynb) , [Repositories](../../../../Repository.ipynb)